<a href="https://colab.research.google.com/github/elaineortsac3/projetos/blob/master/Governanca_Elaine_de_Castro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Atividade Fundamentos de Governança de Dados
- Aluna: Elaine Santos de Castro
- email: elaineortsac3@gmail.com

Leitura Base de Dados

In [6]:
import pandas as pd
import re


In [4]:
df = pd.read_csv('/content/base_clientes_megamart.csv')
display(df.head())

,ID Cliente,Nome Completo,CPF,E-mail,Estado,Data Nascimento,Telefone,Última Compra,Valor Compra,Status Cliente
0,1,Ana Lívia Nascimento,NaN,enzo-gabrielduarte@vieira.com,MA,19/01/1950,+55 (041) 5322 5412,2025-06-10,452.96,Inativo
1,2,NATÁLIA LIMA,NaN,sophiamoraes@uol.com.br,ES,05/01/1946,11 4797 8007,2025-07-20,2109.21,Inativo
2,3,Daniela Souza,944962,lopesmaria-vitoria@gmail.com,PA,11/05/1963,+55 11 2219 5671,2025-06-05,529.01,Inativo
3,4,Joana Porto,845.067.913-39,zdias@souza.com,RS,26/09/1980,+55 21 5062 8931,2025-02-25,2769.16,Bloqueado
4,5,Diego Moura,570.413.692-34,leandrofreitas@barbosa.com,Espírito Santo,07/02/1946,+55 84 6142-4474,2024-09-11,1179.30,Ativo


In [5]:
dados_faltantes = df.isnull().sum()
display(dados_faltantes)

,0
ID Cliente,0
Nome Completo,0
CPF,202
E-mail,0
Estado,0
Data Nascimento,0
Telefone,296
Última Compra,0
Valor Compra,0
Status Cliente,0


In [8]:
def email_valido(email):
    if pd.isna(email):
        return False
    padrao = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return bool(re.match(padrao, email))

email_invalido = ~df['E-mail'].apply(email_valido)
num_email_invalido = email_invalido.sum()
print(f"Quantidade email invalido: {num_email_invalido}")

Quantidade email invalido: 98


In [9]:
def cpf_valido(cpf):
    if not isinstance(cpf, str):
        return False

    cpf_digitos = ''.join(filter(str.isdigit, cpf))

    if len(cpf_digitos) != 11:
        return False

    # Validar CPF
    if cpf_digitos in [str(i) * 11 for i in range(10)]:
        return False

    for i in range(9, 11):
        valor = sum((int(cpf_digitos[num]) * ((i + 1) - num) for num in range(i)))
        digito = ((valor * 10) % 11) % 10
        if digito != int(cpf_digitos[i]):
            return False

    return True

CPF_invalido = ~df['CPF'].apply(cpf_valido)
num_CPF_invalido = CPF_invalido.sum()

print(f"Qtde CPF invalidos: {num_CPF_invalido}")

Qtde CPF invalidos: 297


a) Mapear os problemas da base de dados, segundo dimensões de qualidade:
-- Respostas:
- Completude: Colunas CPF e telefone contém dados ausentes. (Acredito ser melhor identificar e solicitar a correção dos dados)
- Unicidade: Existem registros duplicados por ID CLientte e CPF. (Removerei as linhas duplicadas)
- Consistência: Erros na coluna de Estados, porque tem nomes completos de Estado e as vezes codigo ou siglas. A coluna de nomes tem dados com letras maíusculas e minusculas. Também tem erro nas colunas de CPF e telefone, tem parenteses, pontos e hifens. Também tem CPF com apenas 6 digitos. (Para Estados seria melhor usar siglas e CPF e telefone deveriam ficar apenas com numeros ao invés de simbolos).
- Acuracidade e Validade: Data de nascimento e ultima compra estavam formatadas como texto, tinha um CPF com  6 digitos. (converter as datas para datetime e solicitar correção do CPF).



b) Proposta de contrato de dados para garantir a qualidade dos dados:

Resposta:

- Garantir a padronização dos dados, correção dos dados existentes e limpeza automatizada de pequenas inconsistências;
- Padronizar a coluna Estado: Utilizar sigla de 2 letras
- Limpeza do campo Nome: Converter nomes para formato Primeira letra maiuscula e o restante minuscula
- Tratamento de CPF e telefone: Remover caracteres não numericos, CPF com menos de 11 digitos devem ser revisados, solicitar correção
- Converter datas: Datas devem ser padronizadas para formato datetime
- Remover registros duplicados: Verificar quantidade de linhas duplicadas e remover.

Para garantir a qualidade dos dados de forma contínua, proponho um Contrato de Dados entre as áreas de Vendas fisicas e online e BI, com as seguintes clausulas:

Esquema e estrutura dos dados:
- Definição: As colunas deverão ter tipos e formatos bem definidos, exemplo, ID cliente: inteiro, CPF com 11 digitos, datas com formato datetime. Validar a entrada de dados para garantir que o formato esteja correto antes de inserido;
- Qualidade e Validação dos dados:
- Unicidade: ID cliente e CPF serão campos unicos, novas informações que violem esse criterio serão rejeitados.
- Completude: Os campos de CPF e Telefone devem ser criticos e monitorados. Alertas serão emitidos caso a taxa de preenchimento fique baixa.
- Consitencia: Manter um dicionario de dados para garantir que dados estejam padronizados, como as siglas dos Estados
- Acuracidade: Para evitar problemas com digitação de CPF por exemplo, estes devem ter exatamente 11 digitos para serem aceitos.


c) Descrever as etapas dos ciclos de vida dos dados, mostrar riscos e controles necessários

Respostas:

- Vamos gerenciar o ciclo de vida dos dados, desde a coleta até o descarte dos dados, gerenciar para garantir a qualidade e segurança.
- Coleta de dados - Risco: Inserção de dados duplicados e incompletos com formato inválido. - COntrole: Validar os dados na fonte (podemos criar formularios para cadastro com regras) e utilizar o contrato de dados para guiar o preenchimento dos dados.
- Armazenamento e processamento - Risco: Falta de padronização, atraso na atualização da base de dados, exposição de dados pessoais. - Controle: Automatizar o processo de extração de dados para entrega pontual. Criar codigos para padronizar rotinas de limpeza dos dados. Dados sensiveis como CPF e Telefone podem ser mascarados em relatorios utilizados por pessoas sem permissão de visualização.
- Analise dos dados  - Riscos: Relatorios inconsistentes devido a utilização de diferentes bases de dados ou dados não tratados. - Controle: Utilizar a base de dados tratada e documentar todas as alterações.
- Arquivamento e Exclusão: Risco: Manter dados desnecessários, violando as regras da LGPD. - COntrole: Definir politica interna de retenção dos dados e um processo seguro para descarte, com as devidas autorizações.


d) Atribuir papéis e responsabilidades:

Respostas:

- Atribuir os papéis e responsabilidades:
- Data Owner: Responsavel: Diretor de vendas - Função: definir politicas e requisitos de qualidade para os dados gerados da sua area. Responsavel pela acuracia e relevancia dos dados.
- Data Steward: Responsavel: Gerente de BI - Função: deve implementar e monitorar as regras de qualidade dos dados, garantir a padronização e consistência.
- Data Producer: Responsavel: Equipe de cadastro e sistemas de vendas (fisica e online); Função: Coletar e inserir dados. Deve seguir as regras de validações do contrato de dados
- Data Consumer: Responsavel: Analistas de BI, Marketing e Negócios. Função: utilizar os dados para gerar relatorios e devem reportar os problemas da base de dados.
